# Fixed Effects Estimation (Lab 3)

### Intro and objectives


### In this lab you will learn:
1. examples of fixed effects estimation
2. how to fit fixed effects models in Python


## What I hope you'll get out of this lab
* The feeling that you'll "know where to start" when you need to fit fixed effects models
* Worked Examples
* How to interpret the results obtained

In [ ]:
!pip install wooldridge
!pip install linearmodels
import wooldridge as woo
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import linearmodels as plm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Example. What drives house rental rates ?

####  The data for the years 1980 and 1990 include rental prices and other variables for college towns. The idea is to see whether a stronger presence of students affects rental rates.

#### The idea is to see whether a stronger presence of students affects rental rates.

#### Variables:

city: city label, 1 to 64

year: 80 or 90

pop: city population

enroll: # college students enrolled

rent: average rent

rnthsg: renter occupied units

tothsg: occupied housing units

avginc: per capita income

lenroll: log(enroll)

lpop: log(pop)

lrent: log(rent)

ltothsg: log(tothsg)

lrnthsg: log(rnthsg)

lavginc: log(avginc)

clenroll: change in lrent from 80 to 90

clpop: change in lpop

clrent: change in lrent

cltothsg: change in ltothsg

clrnthsg: change in lrnthsg

clavginc: change in lavginc

pctstu: percent of population students

cpctstu: change in pctstu

y90: =1 if year == 90






In [ ]:
Rental = woo.dataWoo('rental')


In [ ]:
Rental.head()

,city,year,pop,enroll,rent,rnthsg,tothsg,avginc,lenroll,lpop,...,lavginc,clenroll,clpop,clrent,cltothsg,clrnthsg,clavginc,pctstu,cpctstu,y90
0,1,80,75211.0,15303.0,197,13475.0,26167.0,11537.0,9.635804,11.228053,...,9.353314,NaN,NaN,NaN,NaN,NaN,NaN,20.346758,NaN,0
1,1,90,77759.0,18017.0,342,15660.0,29467.0,19568.0,9.799071,11.261370,...,9.881651,-15293.201172,0.033317,0.551607,0.118772,0.150273,0.528337,23.170309,2.823551,1
2,2,80,106743.0,22462.0,323,14580.0,37277.0,19841.0,10.019580,11.578179,...,9.895506,NaN,NaN,NaN,NaN,NaN,NaN,21.043066,NaN,0
3,2,90,141865.0,29769.0,496,26895.0,55540.0,31885.0,10.301223,11.862631,...,10.369891,-22451.699219,0.284451,0.428924,0.398727,0.612289,0.474385,20.984034,-0.059032,1
4,3,80,36608.0,11847.0,216,7026.0,13482.0,11455.0,9.379830,10.508022,...,9.346182,NaN,NaN,NaN,NaN,NaN,NaN,32.361778,NaN,0


In [ ]:
Rental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   city      128 non-null    int64  
 1   year      128 non-null    int64  
 2   pop       128 non-null    float64
 3   enroll    128 non-null    float64
 4   rent      128 non-null    int64  
 5   rnthsg    128 non-null    float64
 6   tothsg    128 non-null    float64
 7   avginc    128 non-null    float64
 8   lenroll   128 non-null    float64
 9   lpop      128 non-null    float64
 10  lrent     128 non-null    float64
 11  ltothsg   128 non-null    float64
 12  lrnthsg   128 non-null    float64
 13  lavginc   128 non-null    float64
 14  clenroll  64 non-null     float64
 15  clpop     64 non-null     float64
 16  clrent    64 non-null     float64
 17  cltothsg  64 non-null     float64
 18  clrnthsg  64 non-null     float64
 19  clavginc  64 non-null     float64
 20  pctstu    128 non-null    float6

#### where pop is city population, avginc is average income, and pctstu is student population as a percentage of city population (during the school year).


A data.frame with 128 observations on 23 variables:

city: city label, 1 to 64

year: 80 or 90

pop: city population

enroll: # college students enrolled

rent: average rent

rnthsg: renter occupied units

tothsg: occupied housing units

avginc: per capita income

lenroll: log(enroll)

lpop: log(pop)

lrent: log(rent)

ltothsg: log(tothsg)

lrnthsg: log(rnthsg)

lavginc: log(avginc)

clenroll: change in lrent from 80 to 90

clpop: change in lpop

clrent: change in lrent

cltothsg: change in ltothsg

clrnthsg: change in lrnthsg

clavginc: change in lavginc

pctstu: percent of population students

cpctstu: change in pctstu

y90: =1 if year == 90


In [ ]:
RentalMultiIndex = Rental.set_index(['city', 'year'], drop=False)

In [ ]:
RentalMultiIndex.head(10)

city  year       pop   enroll  rent   rnthsg   tothsg   avginc  \
city year                                                                   
1    80       1    80   75211.0  15303.0   197  13475.0  26167.0  11537.0   
     90       1    90   77759.0  18017.0   342  15660.0  29467.0  19568.0   
2    80       2    80  106743.0  22462.0   323  14580.0  37277.0  19841.0   
     90       2    90  141865.0  29769.0   496  26895.0  55540.0  31885.0   
3    80       3    80   36608.0  11847.0   216   7026.0  13482.0  11455.0   
     90       3    90   42099.0  10265.0   351   9557.0  16894.0  21202.0   
4    80       4    80   36640.0  13825.0   267   7896.0  14041.0  14682.0   
     90       4    90   46209.0  18173.0   588  10617.0  17926.0  29044.0   
5    80       5    80   62134.0   8175.0   475   5796.0  21337.0  31300.0   
     90       5    90  110330.0  18205.0   925  15112.0  40257.0  56307.0   

             lenroll       lpop  ...    lavginc      clenroll     clpop  \
city year                        ...                                      
1    80     9.635804  11.228053  ...   9.353314           NaN       NaN   
     90     9.799071  11.261370  ...   9.881651 -15293.201172  0.033317   
2    80    10.019580  11.578179  ...   9.895506           NaN       NaN   
     90    10.301223  11.862631  ...  10.369891 -22451.699219  0.284451   
3    80     9.379830  10.508022  ...   9.346182           NaN       NaN   
     90     9.236495  10.647779  ...   9.961851 -11837.763672  0.139757   
4    80     9.534234  10.508896  ...   9.594378           NaN       NaN   
     90     9.807693  10.740930  ...  10.276567 -13815.192383  0.232034   
5    80     9.008836  11.037048  ...  10.351374           NaN       NaN   
     90     9.809451  11.611231  ...  10.938574  -8165.190430  0.574183   

             clrent  cltothsg  clrnthsg  clavginc     pctstu   cpctstu  y90  
city year                                                                    
1    80         NaN       NaN       NaN       NaN  20.346758       NaN    0  
     90    0.551607  0.118772  0.150273  0.528337  23.170309  2.823551    1  
2    80         NaN       NaN       NaN       NaN  21.043066       NaN    0  
     90    0.428924  0.398727  0.612289  0.474385  20.984034 -0.059032    1  
3    80         NaN       NaN       NaN       NaN  32.361778       NaN    0  
     90    0.485508  0.225603  0.307656  0.615669  24.383001 -7.978777    1  
4    80         NaN       NaN       NaN       NaN  37.731987       NaN    0  
     90    0.789478  0.244270  0.296101  0.682190  39.327835  1.595848    1  
5    80         NaN       NaN       NaN       NaN  13.157047       NaN    0  
     90    0.666479  0.634841  0.958321  0.587200  16.500498  3.343451    1  

[10 rows x 23 columns]

In [ ]:
# FIRST FE model estimation:
reg1 = plm.PanelOLS.from_formula(
    formula='lrent ~ y90+lpop+lavginc+pctstu+EntityEffects',
    data=RentalMultiIndex, drop_absorbed=True)


results1 = reg1.fit()

In [ ]:
results1

Dep. Variable:,lrent,R-squared:,0.9765
Estimator:,PanelOLS,R-squared (Between):,0.9391
No. Observations:,128,R-squared (Within):,0.9765
Date:,"Thu, Jan 12 2023",R-squared (Overall):,0.9392
Time:,16:40:04,Log-likelihood,219.27
Cov. Estimator:,Unadjusted,,
,,F-statistic:,624.15
Entities:,64,P-value,0.0000
Avg Obs:,2.0000,Distribution:,"F(4,60)"
Min Obs:,2.0000,,
Max Obs:,2.0000,F-statistic (robust):,624.15


#### We observe, from the previous results, that lpop is not statistically significant(p-value significantly larger than zero: 0.4167, confidence intervals: -0.10:0.249)

### Let's remove lop from the FE model

In [ ]:
# SECOND FE model estimation:
reg2 = plm.PanelOLS.from_formula(
    formula='lrent ~ y90+lavginc+ pctstu+EntityEffects',
    data=RentalMultiIndex, drop_absorbed=True)


results2 = reg2.fit()

In [ ]:
results2

Dep. Variable:,lrent,R-squared:,0.9763
Estimator:,PanelOLS,R-squared (Between):,0.8522
No. Observations:,128,R-squared (Within):,0.9763
Date:,"Thu, Jan 12 2023",R-squared (Overall):,0.8525
Time:,16:40:04,Log-likelihood,218.56
Cov. Estimator:,Unadjusted,,
,,F-statistic:,836.51
Entities:,64,P-value,0.0000
Avg Obs:,2.0000,Distribution:,"F(3,61)"
Min Obs:,2.0000,,
Max Obs:,2.0000,F-statistic (robust):,836.51


### Model interpretation

#### We have fit a model:

$log(rent_{it})=\beta_0+\delta_0*y90_t+\beta_1*log(avcibnc_{it})+\beta_2*pctsu_{it}$


#### 1. Based on the F-statistic (836.1, p-value:0.000) the model is "valid"
#### 2. Based on the R-squared, the model explains 97.6% of the variability.
#### 3. An increment of 1% in avginc leads to 0.31% increment in rent (Bear in mind the log-log relationship)
#### 4. A increment of a unit in pcstu leads to 1% increment in rent (bear in mind the log-level relationship)
#### 5. There has been an increment of 39% in rent prices just due to the passing of time ("time effect")

## We conclude therefore that the main driver of housing rental prices are: average income, percentage of students and "time"